In [1]:
import os
import sys
import math
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import spacy
from tqdm.notebook import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from datasets import load_metric

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using {device}")

using cuda


In [3]:
def get_sent_str(sentence_list):
    sent = " ".join(sentence_list)
    sent = re.sub(r" (?P<punc>[.?,])", r"\1", sent)
    return sent

def get_sent_list(sentences):
    sent_list = []
    for sent in sentences:
        sent_list.append(get_sent_str(sent))
    return sent_list

In [13]:
PRETRAINED_MODEL = 't5-base'
DIR = "question_generator/"
BATCH_SIZE = 1
SEQ_LENGTH = 512
EPOCHS = 200

tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL)
tokenizer.add_special_tokens(
    {'additional_special_tokens': ['<answer>', '<context>']}
)
sacrebleu = load_metric("sacrebleu")
bertscore = load_metric("bertscore")

In [14]:
import importlib
util = importlib.import_module("data.TOEFL-QA.utils")


def set_fuzzy_context(key, raw_data):
    question = [raw_data[key]["question"]]
    results = []
    for ref in get_sent_list(raw_data[key]["sentences"]):
        results.append(bertscore.compute(predictions=question, references=[ref], lang='en'))
    idx = np.argsort(-1 * np.array([i["precision"] for i in results]).ravel())
    top5 = idx[:5]
    sent_list = get_sent_list(raw_data[key]["sentences"])
#         def get_surrounding(sent_list, ind):
#             if ind == 0:
#                 return set_list[i] + sent_list[i+1]
#             elif ind == len(sent_list)-1:
#                 return set_list[i-1] + sent_list[i]
#             else:
#                 return set_list[i-1] + sent_list[i] + sent_list[i+1]
    raw_data[key]["context"] = " ".join([sent_list[i] for i in sorted(top5)]) # reorder sentences

def preprocess(raw_data):
    keys = list(raw_data.keys())
    for key in tqdm(keys):
        raw_data[key]["question"] = get_sent_str(raw_data[key]["question"])
        set_fuzzy_context(key, raw_data)


if "train_processed.npy" in os.listdir():
    train_raw = np.load("train_processed.npy", allow_pickle=True).item()
    dev_raw = np.load("dev_processed.npy", allow_pickle=True).item()
    test_raw = np.load("test_processed.npy", allow_pickle=True).item()
else:
    TOEFL_PATH = "./data/TOEFL-QA/data/"
    raw = util.load_data(TOEFL_PATH)
    train_raw, dev_raw, test_raw = tuple(raw)
    preprocess(dev_raw)
    np.save("dev_processed.npy", dev_raw)
    preprocess(test_raw)
    np.save("test_processed.npy", test_raw)
    preprocess(train_raw)
    np.save("train_processed.npy", train_raw)

In [15]:
train_raw.keys()
def get_sentences_len(key):
    x = []
    for sent in train_raw[key]['sentences']:
        x.extend(sent)
    # tokenized = tokenizer(" ".join(x))
    return len(x)
get_sentences_len('tpo_1-conversation_1_2')

504

### Problem
Context for both race and toefl texts / question contexts are not guaranteed to be <512 tokens, which is expected by our T5 base. Also, they both include lots of info not related to the question, which is the normal expectation in previous QG (ex. SQuAD models)
### Possible solutions
- Annotate.
- Use a metric (bertscore) to find semantically similar sentences to a given answer, and take the top n for a "fuzzy" context

In [16]:
class TOEFLDataset(Dataset):
    def __init__(self, data_dict):
        self.data = data_dict
        self.idx_map = list(data_dict.keys())       

    def __len__(self):
         return len(self.idx_map)

    def __getitem__(self, idx):   
        row = self.data[self.idx_map[idx]]  

        encoded_text = tokenizer(
            row['context'], 
            padding=True,
            max_length=SEQ_LENGTH,
            truncation=True,
            return_tensors="pt"
        )
        encoded_text['input_ids'] = torch.squeeze(encoded_text['input_ids'])
        encoded_text['attention_mask'] = torch.squeeze(encoded_text['attention_mask'])

        encoded_question = tokenizer(
            row['question'],
            padding=True,
            max_length=SEQ_LENGTH,
            truncation=True,
            return_tensors='pt'
        )
        encoded_question['input_ids'] = torch.squeeze(encoded_question['input_ids'])

        return encoded_text.to(device), encoded_question.to(device)

    
train_set = TOEFLDataset(train_raw)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
dev_set = TOEFLDataset(dev_raw)
dev_loader = DataLoader(dev_set, batch_size=BATCH_SIZE)


In [17]:

def train(epoch, best_val_loss):
    model.train()
    total_loss = 0.
    for batch_index, batch in tqdm(enumerate(train_loader)):
        data, target = batch 
        # data, target = (data.to(device), target.to(device))
        optimizer.zero_grad()
        masked_labels = mask_label_padding(target['input_ids'])
        output = model(
            input_ids=data['input_ids'],
            attention_mask=data['attention_mask'],
            labels=masked_labels
        )
        output[0].backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        total_loss += output[0].item()
        if batch_index % 500 == 499:
            print(f'| epoch {epoch} | {batch_index}/{len(train_loader)} batches | loss {total_loss / 500}')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_loss': best_val_loss,
            }, "question_generator/toeflqa_finetune_tmp.pth")
            total_loss = 0
def valid(epoch):
    model.eval()
    total_loss = 0.
    for batch_index, batch in tqdm(enumerate(dev_loader)):
        data, target = batch 
        # data, target = (data.to(device), target.to(device))
        optimizer.zero_grad()
        masked_labels = mask_label_padding(target['input_ids'])
        output = model(
            input_ids=data['input_ids'],
            attention_mask=data['attention_mask'],
            labels=masked_labels
        )
        total_loss += output[0].item()
    return total_loss / len(dev_loader)
        
def mask_label_padding(labels):
    MASK_ID = -100
    labels[labels==tokenizer.pad_token_id] = MASK_ID
    return labels


In [18]:
config = T5Config(decoder_start_token_id=tokenizer.pad_token_id) # eos
model = T5ForConditionalGeneration(config).from_pretrained('t5-base')
model.resize_token_embeddings(len(tokenizer)) # to account for new special tokens
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [19]:
best_val_loss = float("inf")

for epoch in range(1, EPOCHS + 1):

    train(epoch, best_val_loss)
    torch.cuda.empty_cache()
    val_loss = valid(model)
    torch.cuda.empty_cache()
    print(f'\nend of epoch {epoch}\n valid loss: {val_loss}\n')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_loss': best_val_loss,
        }, "question_generator/toeflqa_finetune.pth")
        print("Model saved.\n")


0it [00:00, ?it/s]

| epoch 1 | 499/717 batches | loss 4.510052051305771


0it [00:00, ?it/s]


end of epoch 1
 valid loss: 4.156029810828548

Model saved.



0it [00:00, ?it/s]

...


end of epoch 199
 valid loss: 1.6725356840798933

Model saved.



0it [00:00, ?it/s]

| epoch 200 | 499/717 batches | loss 1.7201947879940271


0it [00:00, ?it/s]


end of epoch 200
 valid loss: 1.6712973372590156

Model saved.

